# Experiments for Higher-Order Network Statistics vs Cognitive Performance

In [2]:
# load all third part libraries
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import bct
import scipy as sp
import scipy.stats
import networkx as nx

In [3]:
# load networks
mplex2 = np.load('/Users/nick/desktop/mplex/data/map/mplex300_partial.npy') # normalized already
#mplex2 = mplex2.reshape(820,300,300)
#print mplex_orig.shape
mplex1 = np.load('/Users/nick/desktop/mplex/data/map/mplex300.npy') # shape = (820,300,300) # not normalized
mplex1_unnorm = mplex1.copy()
# load subject id's
#subjects = np.loadtxt('/Users/nick/desktop/mplex/data/subjectIDs.txt',dtype=np.int64) # shape = (820,)
# load cogntive data
cog_head, cog_vals = np.load('/Users/nick/desktop/mplex/data/demo.npy')

In [4]:
for i in range(820):
    mplex1[i,:,:] = (mplex1[i,:,:] - np.mean(mplex1[i,:,:]))/np.std(mplex1[i,:,:])
    np.fill_diagonal(mplex1[i,:,:],0)

In [5]:
thresh = 3
mp0 = mplex1.copy()
mp0[mp0 < thresh] = 0
mp0[mp0 >= thresh] = 1
for i in range(820): np.fill_diagonal(mp0[i,:,:],0)

mp1 = mplex1.copy()
mp1[mp1 < thresh] = 0
for i in range(820): np.fill_diagonal(mp1[i,:,:],0)
    
mp2 = mplex1.copy()
mp2[mp2 < 0] = 0

mp3 = mplex2.copy()
mp3[mp3 < thresh] = 0
mp3[mp3 >= thresh] = 1
for i in range(820): np.fill_diagonal(mp3[i,:,:],0)
    
mp4 = mplex2.copy()
mp4[mp4 < thresh] = 0
for i in range(820): np.fill_diagonal(mp4[i,:,:],0)
    
mp5 = mplex2.copy()
mp5[mp5 < 0] = 0

mp_list = [mp0,mp1,mp2,mp3,mp4,mp5]

def z_norm(x):
    """
    Normalize values to be z-scores
    """
    for i in range(820):
        x[i,:,:] = (x[i,:,:] - np.mean(x[i,:,:])) / np.std(x[i,:,:])
    x[x<0] = 0
    return x

## for extended testing
mp6 = mplex1_unnorm.copy()
mp6[mp6<0] = 0
mp6 = z_norm(mp6)

mp9 = mplex1_unnorm.copy()
mp9[mp9<0.15] = 0
mp9 = z_norm(mp9)

mp10 = mplex1_unnorm.copy()
mp10[mp10<0.3] = 0
mp10 = z_norm(mp10)

mp11 = mplex1_unnorm.copy()
mp11[mp11<0.45] = 0
mp11 = z_norm(mp10)

mp12 = mplex1_unnorm.copy()
mp12[mp12<0.6] = 0
mp12 = z_norm(mp12)

mp13 = mplex2.copy()
mp13[mp13 < 4.5] = 0
mp13[mp13 >= 4.5] = 1
for i in range(820): np.fill_diagonal(mp13[i,:,:],0)

mp14 = mplex2.copy()
mp14[mp14 < 6] = 0
mp14[mp14 >= 6] = 1
for i in range(820): np.fill_diagonal(mp14[i,:,:],0)
    
mp15 = mplex2.copy()
mp15[mp15 < 9] = 0
mp15[mp15 >= 9] = 1
for i in range(820): np.fill_diagonal(mp15[i,:,:],0)


mp_list_extended = [mp0,mp1,mp2,mp3,mp4,mp5,mp6,mp9,mp10,mp11,mp12,mp13,mp14,mp15]

Mplex Code:
    - mp1 : full correlation, normalized, binary thresholded
    - mp2 : full correlation, normalized, thresholded
    - mp3 : full correlation, normalized, thresholded at 0
    - mp4 : partial correlation, normalized, binary thresholded
    - mp5 : partial correaltion, normalized, thresholded
    - mp6 : partial correlation, normalized, thresholded at 0


Cognitive data pre-processing and function for testing correlation between cognitive data and network metrics:

In [22]:
def str_to_int(x):
    """
    Convert array of strings to their integer indices
    """
    try:
        y = x.astype(np.float32)
    except ValueError:
        x = x.astype('str')
        unique_vals = np.unique(x)
        y = np.zeros(len(x),dtype=np.int32)
        for i, val in enumerate(x):
            y[i] = np.where(val==unique_vals)[0][0]
    return y

def test_corr(cog_vec, net_vec):
    """
    Test correlation between cognitive values (cog_vec)
    and network statistic values (net_vec). Each individual
    should have a single value for each - i.e. len = 820
    """
    # create mask
    midx = np.where(cog_vec=='nan')
    mask = np.ones(820,dtype='bool')
    mask[midx] = 0
    # convert to int
    cog_vec = str_to_int(cog_vec)
    # mask
    cog_vec = cog_vec[mask]
    net_vec = net_vec[mask]
    # normalize cog vals
    cog_vec = (cog_vec - np.mean(cog_vec)) / (np.std(cog_vec)+1e-7)
    net_vec = (net_vec - np.mean(net_vec)) / (np.std(net_vec)+1e-7)
    # test
    r,p = sp.stats.pearsonr(cog_vec,net_vec)
    return r,p

## 1. Centrality

Here, we test full (mplex1) and partial (mplex2) correlation networks using various centrality measures. We perform this analysis for binary, thresholded normalized, and unthresholded normalized versions of each network.

For each metric that returns a vector of values for each individual network, we will take the variance, the max, and the mean.

### 1A. Node Betweenness

In [58]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn = bct.betweenness_bin(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  1  Col:  Age  Corr:  -0.153563272133 Pval:  1.00127260715e-05
MPLEX:  0  TRANS:  1  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  0.1835799272 Pval:  1.19692524201e-07
MPLEX:  0  TRANS:  1  Col:  FS_L_Cerebellum_Cort_Vol.1  Corr:  0.175765355157 Pval:  4.08492768955e-07
MPLEX:  0  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.219606405234 Pval:  0.0364781631871
MPLEX:  0  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.182538838488 Pval:  0.083305565637
MPLEX:  0  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -0.199794668538 Pval:  0.0575954754023
MPLEX:  0  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.173748028582 Pval:  0.0681937100548
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  1  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  0.174809001135 Pval:  4.72972035516e-07
MPLEX:  1  TRANS:  1  Col:  FS_L_Cerebellum_Cort

### 1B. PageRank

In [62]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn = bct.pagerank_centrality(net,0.85)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  Age  Corr:  0.164148861315 Pval:  2.29633237565e-06
MPLEX:  0  TRANS:  0  Col:  Alpha_Peak  Corr:  0.231307301148 Pval:  0.0256927983419
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  0  Col:  Age  Corr:  0.175611401453 Pval:  4.18267651325e-07
MPLEX:  1  TRANS:  0  Col:  MEGsession_Scans  Corr:  0.156958807135 Pval:  0.1308414843
MPLEX:  1  TRANS:  0  Col:  Alpha_Peak  Corr:  0.256179972317 Pval:  0.0131895975615
MPLEX:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.173073801179 Pval:  0.0692877783711
Mplex :  2
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  2  TRANS:  0  Col:  Age  Corr:  0.190356529418 Pval:  3.9523457728e-08
MPLEX:  2  TRANS:  2  Col:  MEGsession_Label  Corr:  0.194533281252 Pval:  0.0602682409343
MPLEX:  2  TRANS:  0  Col:  PSQI_BedTime  Corr:  0.153524548987 Pval:  1.0065048682e-05
MPLEX:  2

### 1C. Eigenvector Centrality

In [63]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn = bct.eigenvector_centrality_und(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  MEGsession_Scans  Corr:  0.205841693015 Pval:  0.046549539495
MPLEX:  0  TRANS:  1  Col:  MEGsession_Scans  Corr:  -0.203648111968 Pval:  0.0489867946786
MPLEX:  0  TRANS:  2  Col:  MEGsession_Scans  Corr:  0.180818866296 Pval:  0.0811496141037
MPLEX:  0  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Random  Corr:  0.221127704679 Pval:  0.0351683744052
MPLEX:  0  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  0.17318523467 Pval:  0.0691059809736
MPLEX:  0  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.17421612347 Pval:  0.0674424033032
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  2  Col:  MEGsession_Label  Corr:  -0.174385751501 Pval:  0.0927595247135
MPLEX:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Random  Corr:  0.185401593698 Pval:  0.0784998170837
MPLEX:  1  TRANS:  1  Col:  Social_Task_TOM_Median_RT_Random  Corr:  -

## 2. Clustering

### 2A. Clustering Coefficienct

In [71]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        ccf = bct.clustering_coef_bu(net)
        nb[0,net_idx] = np.var(ccf)
        nb[1,net_idx] = np.mean(ccf)
        nb[2,net_idx] = np.max(ccf)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  Beta_Peak  Corr:  0.235719206657 Pval:  0.0229297787789
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  0  Col:  Alpha_Peak  Corr:  0.160005330398 Pval:  0.12550879694
MPLEX:  1  TRANS:  0  Col:  Beta_Peak  Corr:  0.175090421736 Pval:  0.0932166803027
MPLEX:  1  TRANS:  0  Col:  FS_L_Cerebellum_WM_Vol  Corr:  -0.150666390054 Pval:  1.47356972784e-05
MPLEX:  1  TRANS:  0  Col:  FS_L_Pallidum_Vol  Corr:  -0.166616453374 Pval:  1.60687362324e-06
MPLEX:  1  TRANS:  0  Col:  FS_L_VentDC_Vol  Corr:  -0.151874048269 Pval:  1.25540504341e-05
MPLEX:  1  TRANS:  0  Col:  FS_R_VentDC_Vol  Corr:  -0.155317162959 Pval:  7.8968135665e-06
MPLEX:  1  TRANS:  0  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.183608393616 Pval:  0.0537346917072
MPLEX:  1  TRANS:  2  Col:  Social_Task_TOM_Median_RT_Unsure  Corr:  -0.239816202181 Pval:  0.0112406205901
Mplex :  2
Net 

### 2B. Transitivity

In [94]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn = bct.transitivity_bu(net)
        else:
            btwn = bct.transitivity_wu(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  1  Col:  FS_BrainSeg_Vol  Corr:  -0.152003389987 Pval:  1.23395516946e-05
MPLEX:  0  TRANS:  2  Col:  FS_BrainSeg_Vol  Corr:  -0.152003389987 Pval:  1.23395516946e-05
MPLEX:  0  TRANS:  1  Col:  FS_LCort_GM_Vol  Corr:  -0.151566123759 Pval:  1.30790913642e-05
MPLEX:  0  TRANS:  2  Col:  FS_LCort_GM_Vol  Corr:  -0.151566123759 Pval:  1.30790913642e-05
MPLEX:  0  TRANS:  1  Col:  FS_RCort_GM_Vol  Corr:  -0.1521042518 Pval:  1.21747109763e-05
MPLEX:  0  TRANS:  2  Col:  FS_RCort_GM_Vol  Corr:  -0.1521042518 Pval:  1.21747109763e-05
MPLEX:  0  TRANS:  1  Col:  FS_TotCort_GM_Vol  Corr:  -0.152581458335 Pval:  1.14228367496e-05
MPLEX:  0  TRANS:  2  Col:  FS_TotCort_GM_Vol  Corr:  -0.152581458335 Pval:  1.14228367496e-05
MPLEX:  0  TRANS:  1  Col:  FS_Total_GM_Vol  Corr:  -0.153580939019 Pval:  9.98894085783e-06
MPLEX:  0  TRANS:  2  Col:  FS_Total_GM_Vol  Corr:  -0.153580939019 Pval:  9.98894085783e-06
M

## 3. Core

### 3A. Assortativity

In [92]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn = bct.assortativity_bin(net,0)
        else:
            btwn = bct.assortativity_wei(net,0)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  2
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  3
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  3  TRANS:  0  Col:  Gender  Corr:  -0.252385851961 Pval:  2.21262608819e-13
MPLEX:  3  TRANS:  0  Col:  FS_BrainSeg_Vol  Corr:  -0.255384475998 Pval:  1.12425901874e-13
MPLEX:  3  TRANS:  0  Col:  FS_BrainSeg_Vol_No_Vent  Corr:  -0.252344003021 Pval:  2.23349307449e-13
MPLEX:  3  TRANS:  0  Col:  FS_BrainSeg_Vol_No_Vent_Surf  Corr:  -0.252444618365 Pval:  2.18364597855e-13
MPLEX:  3  TRANS:  0  Col:  FS_Total_GM_Vol  Corr:  -0.261207631617 Pval:  2.94352282437e-14
MPLEX:  3  TRANS:  0  Col:  FS_Mask_Vol  Corr:  -0.253462601337 Pval:  1.73685623382e-13
MPLEX:  3  TRANS:  0  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  -0.303716374263 Pval:  5.85251544264e-19
MPLEX:  3  TRANS:  0  Col:  FS_L_Cerebellum_Cort_Vol.1  

### 3B. Rich Club --  GOOD RESULTS HERE, DON'T ALTER

In [104]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((20,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn,_,_ = bct.rich_club_bu(net,20)
        else:
            btwn = bct.rich_club_wu(net,20)
        for ii in range(20):
            nb[ii,net_idx] = btwn[ii]
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0


//anaconda/lib/python2.7/site-packages/bctpy-0.4.1-py2.7.egg/bct/algorithms/core.py:354: RuntimeWarning: invalid value encountered in double_scalars
  R[k] = Ek[k] / (Nk[k] * (Nk[k] - 1))


Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  Acquisition  Corr:  -0.152915788908 Pval:  1.09226587783e-05
MPLEX:  0  TRANS:  0  Col:  Age  Corr:  0.163937460684 Pval:  2.36707869456e-06
MPLEX:  0  TRANS:  1  Col:  Age  Corr:  0.169223339535 Pval:  1.09573142996e-06
MPLEX:  0  TRANS:  2  Col:  MEGsession_Label  Corr:  0.160168560476 Pval:  0.123052161489
MPLEX:  0  TRANS:  3  Col:  MEGsession_Label  Corr:  0.162458208572 Pval:  0.117717020536
MPLEX:  0  TRANS:  5  Col:  MEGsession_Label  Corr:  0.1669080359 Pval:  0.107860993106
MPLEX:  0  TRANS:  6  Col:  MEGsession_Label  Corr:  0.154272628919 Pval:  0.137644697164
MPLEX:  0  TRANS:  0  Col:  Alpha_Peak  Corr:  0.164600395026 Pval:  0.114874993273
MPLEX:  0  TRANS:  1  Col:  Alpha_Peak  Corr:  0.155633139438 Pval:  0.13631208263
MPLEX:  0  TRANS:  11  Col:  Alpha_Peak  Corr:  -0.168906867464 Pval:  0.105554790683
MPLEX:  0  TRANS:  11  Col:  Beta_Peak  Corr:  -0.180556364691 Pval:  0.0832846124005
MPLEX:  0

//anaconda/lib/python2.7/site-packages/bctpy-0.4.1-py2.7.egg/bct/algorithms/core.py:448: RuntimeWarning: invalid value encountered in double_scalars
  Rw[k] = Wr / np.sum(wrank_r)


Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  10  Col:  Alpha_Peak  Corr:  0.192731506112 Pval:  0.0641851613114
MPLEX:  1  TRANS:  11  Col:  Alpha_Peak  Corr:  0.194155404923 Pval:  0.0622050639616
MPLEX:  1  TRANS:  12  Col:  Alpha_Peak  Corr:  0.164310589016 Pval:  0.115524361027
MPLEX:  1  TRANS:  10  Col:  Beta_Peak  Corr:  0.188976638429 Pval:  0.0696530099306
MPLEX:  1  TRANS:  11  Col:  Beta_Peak  Corr:  0.152182446806 Pval:  0.145325664722
MPLEX:  1  TRANS:  8  Col:  FS_IntraCranial_Vol  Corr:  -0.151699316791 Pval:  1.28494681122e-05
MPLEX:  1  TRANS:  8  Col:  FS_BrainSeg_Vol  Corr:  -0.160779140031 Pval:  3.70776591218e-06
MPLEX:  1  TRANS:  9  Col:  FS_BrainSeg_Vol  Corr:  -0.156461813136 Pval:  6.75390106881e-06
MPLEX:  1  TRANS:  10  Col:  FS_BrainSeg_Vol  Corr:  -0.161236716547 Pval:  3.47620003798e-06
MPLEX:  1  TRANS:  8  Col:  FS_BrainSeg_Vol_No_Vent  Corr:  -0.156228915562 Pval:  6.97282201176e-06
MPLEX:  1  TRANS:  9  Col:  FS_BrainSeg_Vol_No_Vent

In [107]:
# extended mplex list
for mp_idx in range(5,len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((20,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn,_,_ = bct.rich_club_bu(net,20)
        else:
            btwn = bct.rich_club_wu(net,20)
        for ii in range(20):
            nb[ii,net_idx] = btwn[ii]
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  5
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  6
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  7
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  7  TRANS:  1  Col:  Toolbox_Compl  Corr:  -0.155204985024 Pval:  8.01825732434e-06
MPLEX:  7  TRANS:  2  Col:  Toolbox_Compl  Corr:  -0.159252710141 Pval:  4.59178583773e-06
MPLEX:  7  TRANS:  3  Col:  Toolbox_Compl  Corr:  -0.161800029301 Pval:  3.21010022519e-06
MPLEX:  7  TRANS:  4  Col:  Toolbox_Compl  Corr:  -0.160920638186 Pval:  3.63462590604e-06
MPLEX:  7  TRANS:  5  Col:  Toolbox_Compl  Corr:  -0.161592001179 Pval:  3.30601467378e-06
MPLEX:  7  TRANS:  6  Col:  Toolbox_Compl  Corr:  -0.157893873546 Pval:  5.54535239795e-06
MPLEX:  7  TRANS:  7  Col:  Toolbox_Compl  Corr:  -0.157783142479 Pval:  5.63088892469e-06
MPLEX:  7  TRANS:  8  Col:  Toolbox_Compl  Corr:  -0.156941881504 Pval:  6.32316925156e-06
MPLEX:  7  TRANS:  9  Col:  Toolbox_Compl  Corr:  -0.159315158915

## 4. Degree

In [108]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((3,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        if mp_idx == 0 or mp_idx == 3:
            btwn = bct.degrees_und(net)
        else:
            btwn = bct.strengths_und(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.max(btwn)  
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  2  Col:  MEGsession_Label  Corr:  0.154944941186 Pval:  0.135917299933
MPLEX:  0  TRANS:  2  Col:  FS_LCort_GM_Vol  Corr:  -0.152450350112 Pval:  1.16248780522e-05
MPLEX:  0  TRANS:  2  Col:  FS_TotCort_GM_Vol  Corr:  -0.150464596856 Pval:  1.51337395458e-05
MPLEX:  0  TRANS:  0  Col:  FS_SubCort_GM_Vol  Corr:  -0.150322191257 Pval:  1.54207742917e-05
MPLEX:  0  TRANS:  2  Col:  FS_SubCort_GM_Vol  Corr:  -0.153811305759 Pval:  9.68368335401e-06
MPLEX:  0  TRANS:  0  Col:  FS_Total_GM_Vol  Corr:  -0.155605263447 Pval:  7.5929067431e-06
MPLEX:  0  TRANS:  2  Col:  FS_Total_GM_Vol  Corr:  -0.155652514909 Pval:  7.54414082859e-06
MPLEX:  0  TRANS:  0  Col:  FS_L_Pallidum_Vol  Corr:  -0.165911942535 Pval:  1.78025391388e-06
MPLEX:  0  TRANS:  2  Col:  FS_L_Hippo_Vol  Corr:  -0.161205958417 Pval:  3.49132157145e-06
MPLEX:  0  TRANS:  1  Col:  FS_L_ChoroidPlexus_Vol  Corr:  -0.159202324028 Pval:  4.6241557

## 5. Distance

### 5A. Characteristic Path Length

In [111]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn,_,_,_,_ = bct.charpath(net+1e-7)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  FS_L_ChoroidPlexus_Vol  Corr:  -0.159202324028 Pval:  4.62415574341e-06
MPLEX:  0  TRANS:  0  Col:  FS_R_ChoroidPlexus_Vol  Corr:  -0.161664703779 Pval:  3.27218635765e-06
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  0  Col:  FS_L_ChoroidPlexus_Vol  Corr:  -0.164300997134 Pval:  2.246678119e-06
MPLEX:  1  TRANS:  0  Col:  FS_R_ChoroidPlexus_Vol  Corr:  -0.158092334822 Pval:  5.39514436744e-06
Mplex :  2
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  2  TRANS:  0  Col:  FS_IntraCranial_Vol  Corr:  0.177124875452 Pval:  3.312106401e-07
MPLEX:  2  TRANS:  0  Col:  FS_BrainSeg_Vol  Corr:  0.152308409174 Pval:  1.18474497555e-05
MPLEX:  2  TRANS:  0  Col:  FS_BrainSeg_Vol_No_Vent  Corr:  0.155317822182 Pval:  7.89610510664e-06
MPLEX:  2  TRANS:  0  Col:  FS_BrainSeg_Vol_No_Vent_Surf  Corr:  0.155015410071 Pval:  8.22755176905e-06
MPLE

### 5B. Efficiency from CharPath

In [112]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,btwn,_,_,_ = bct.charpath(net+1e-7)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  FS_L_ChoroidPlexus_Vol  Corr:  0.159202324028 Pval:  4.62415574341e-06
MPLEX:  0  TRANS:  0  Col:  FS_R_ChoroidPlexus_Vol  Corr:  0.161664703779 Pval:  3.27218635765e-06
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  0  Col:  FS_L_ChoroidPlexus_Vol  Corr:  0.159202324242 Pval:  4.62415560598e-06
MPLEX:  1  TRANS:  0  Col:  FS_R_ChoroidPlexus_Vol  Corr:  0.161664703783 Pval:  3.27218635586e-06
Mplex :  2
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  3
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  3  TRANS:  0  Col:  Release  Corr:  -0.210872823383 Pval:  1.07427188892e-09
MPLEX:  3  TRANS:  0  Col:  Acquisition  Corr:  -0.217278102296 Pval:  3.2226025393e-10
MPLEX:  3  TRANS:  0  Col:  Age  Corr:  0.208706885641 Pval:  1.60041663446e-09
MPLEX:  3  TRANS:  0  Col:  fMRI_3T_ReconVrs  Corr:  -0.169472305481 Pval: 

### 5C. Global Efficiency

In [114]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn,_ = bct.efficiency_bin(net)
        nb[0,net_idx] = btwn
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.155007838619 Pval:  0.137913150612
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.155007838619 Pval:  0.137913150612
Mplex :  2
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
Mplex :  3
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  3  TRANS:  0  Col:  Release  Corr:  0.164750880329 Pval:  2.10579390104e-06
MPLEX:  3  TRANS:  0  Col:  Acquisition  Corr:  0.170996215393 Pval:  8.41719757456e-07
MPLEX:  3  TRANS:  0  Col:  Age  Corr:  -0.16980304746 Pval:  1.00549971078e-06
MPLEX:  3  TRANS:  0  Col:  Toolbox_Compl  Corr:  -0.170602239542 Pval:  8.92734542824e-07
MPLEX:  3  TRANS:  0  Col:  FS_IntraCranial_Vol  Corr:  0.160692896826 Pval:  3.75303336091e-06
MPLEX:  3  TRANS:  0  Col:  FS_LCort_GM_Vol  Corr:  0.173618028805 Pval:  5.67046383531e-07
MPLEX:  3  TRANS:  0  C

### 5D. Local Efficiency - IMPORTANT RESULTS IN MPLEX11

In [115]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((6,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,btwn = bct.efficiency_bin(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.std(btwn)
        nb[3,net_idx] = np.max(btwn)
        nb[4,net_idx] = np.min(btwn)
        nb[5,net_idx] = np.max(btwn) - np.min(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  1  Col:  Alpha_Peak  Corr:  -0.155007838619 Pval:  0.137913150612
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  1  Col:  Alpha_Peak  Corr:  -0.155007838619 Pval:  0.137913150612
Mplex :  2
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  2  TRANS:  0  Col:  Beta_Peak  Corr:  -0.155274598691 Pval:  0.137228383055
MPLEX:  2  TRANS:  2  Col:  Beta_Peak  Corr:  -0.155515305119 Pval:  0.136612710735
Mplex :  3
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  3  TRANS:  0  Col:  Release  Corr:  0.175996919758 Pval:  3.9420336549e-07
MPLEX:  3  TRANS:  1  Col:  Release  Corr:  0.164750880329 Pval:  2.10579390104e-06
MPLEX:  3  TRANS:  2  Col:  Release  Corr:  0.176842719926 Pval:  3.45990897734e-07
MPLEX:  3  TRANS:  0  Col:  Acquisition  Corr:  0.194286441408 Pval:  2.04020297921e-08
MPLEX:  3  TRANS:  1  Col:  Acquisition  Corr: 

### 5E. Eccentricity

In [116]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((6,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,_,btwn,_,_ = bct.charpath(net)
        nb[0,net_idx] = np.var(btwn)
        nb[1,net_idx] = np.mean(btwn)
        nb[2,net_idx] = np.std(btwn)
        nb[3,net_idx] = np.max(btwn)
        nb[4,net_idx] = np.min(btwn)
        nb[5,net_idx] = np.max(btwn) - np.min(btwn)
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  Acquisition  Corr:  0.174411404701 Pval:  5.02569507102e-07
MPLEX:  0  TRANS:  1  Col:  Acquisition  Corr:  0.173950575109 Pval:  5.39106361753e-07
MPLEX:  0  TRANS:  2  Col:  Acquisition  Corr:  0.174158024888 Pval:  5.2235298279e-07
MPLEX:  0  TRANS:  0  Col:  Age  Corr:  -0.186583207265 Pval:  7.36172197204e-08
MPLEX:  0  TRANS:  1  Col:  Age  Corr:  -0.189275701669 Pval:  4.7292246668e-08
MPLEX:  0  TRANS:  2  Col:  Age  Corr:  -0.186245375247 Pval:  7.77851835574e-08
MPLEX:  0  TRANS:  1  Col:  Toolbox_Compl  Corr:  -0.150988082065 Pval:  1.41216851074e-05
MPLEX:  0  TRANS:  0  Col:  Alpha_Peak  Corr:  -0.171442495676 Pval:  0.100349636617
MPLEX:  0  TRANS:  1  Col:  Alpha_Peak  Corr:  -0.150205389741 Pval:  0.150688305041
MPLEX:  0  TRANS:  2  Col:  Alpha_Peak  Corr:  -0.173005006076 Pval:  0.0972437037144
MPLEX:  0  TRANS:  0  Col:  FS_L_Cerebellum_Cort_Vol  Corr:  0.224407955638 Pva

# NOTE: Sections 6 and 7 take a long time (few hours) to run, so they should be run in a cluster instead of a notebook. The exact code used is still here though.


## 6. Modularity

In [127]:
# node betweenness
for mp_idx in range(len(mp_list_extended)):
    mplex = mp_list_extended[mp_idx]
    nb = np.empty((1,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 5 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        _,btwn1 = bct.community_louvain(net)
        nb[0,net_idx] = btwn1
        #nb[1,net_idx] = btwn2
        #nb[2,net_idx] = btwn3
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  5
Net Idx :  10
Net Idx :  15


KeyboardInterrupt: 

## 7. Motifs

### 7A. Binary Structural 3-node Motifs

In [23]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((13,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn,_ = bct.motif3funct_bin(net)
        # there are 13 types of 3-node motifs.
        for ii in range(13):
            nb[ii,net_idx] = btwn[ii]
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

Mplex :  0
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  0  TRANS:  0  Col:  MEGsession_Scans  Corr:  -0.172628410725 Pval:  0.0961490364012
MPLEX:  0  TRANS:  1  Col:  MEGsession_Scans  Corr:  -0.170978233196 Pval:  0.0994198936212
MPLEX:  0  TRANS:  2  Col:  MEGsession_Scans  Corr:  -0.171552047184 Pval:  0.0982727625899
MPLEX:  0  TRANS:  3  Col:  MEGsession_Scans  Corr:  -0.166229992189 Pval:  0.109319862088
MPLEX:  0  TRANS:  5  Col:  MEGsession_Scans  Corr:  -0.163235756076 Pval:  0.115946438835
MPLEX:  0  TRANS:  8  Col:  MEGsession_Scans  Corr:  -0.153468623345 Pval:  0.139732258164
Mplex :  1
Net Idx :  0
Net Idx :  250
Net Idx :  500
Net Idx :  750
MPLEX:  1  TRANS:  0  Col:  MEGsession_Scans  Corr:  -0.172628410725 Pval:  0.0961490364012
MPLEX:  1  TRANS:  1  Col:  MEGsession_Scans  Corr:  -0.170978233196 Pval:  0.0994198936212
MPLEX:  1  TRANS:  2  Col:  MEGsession_Scans  Corr:  -0.171552047184 Pval:  0.0982727625899
MPLEX:  1  TRANS:  3  Col:  MEGsessio

KeyboardInterrupt: 

### 7B. Weighted Structural 3-node Motifs

In [20]:
# node betweenness
for mp_idx in range(len(mp_list)):
    mplex = mp_list[mp_idx]
    nb = np.empty((13,mplex.shape[0]))
    print 'Mplex : ' , mp_idx
    for net_idx in range(mplex.shape[0]):
        if net_idx % 250 == 0: print 'Net Idx : ' ,net_idx
        net = mplex[net_idx,:,:]
        btwn,_ = bct.motif3funct_wei(net)
        # there are 13 types of 3-node motifs.
        for ii in range(13):
            nb[ii,net_idx] = btwn[ii]
    for cog_idx in range(cog_vals.shape[1]):
        for nb_idx in range(nb.shape[0]):
            corr_val,pval = test_corr(cog_vals[:,cog_idx],nb[nb_idx,:])
            if np.abs(corr_val) > 0.15: 
                print 'MPLEX: ',mp_idx,' TRANS: ',nb_idx,' Col: ',cog_head[cog_idx], ' Corr: ',corr_val,'Pval: ',pval

KeyboardInterrupt: 